In [2]:
import requests
import json

import os
import random
import re

import lxml
from lxml import html
from lxml import etree

import pandas as pd
import urllib.parse

import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
file_path = '/Users/yishuzhang/Downloads/Kickstarter_sample.csv'
df = pd.read_csv(file_path)

In [57]:
# get name
df['creator_name'] = df['creator'].apply(lambda x: json.loads(x)['name'])

In [58]:
# get url
df['url_scrape'] = df['urls'].apply(lambda x: json.loads(x)['web']['project'])

In [59]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,state,state_changed_at,static_usd_rate,urls,usd_exchange_rate,usd_pledged,usd_type,video,creator_name,url_scrape
0,53,#NoEstamosAquídeVisita Publicaremos las nuevas...,"{""id"":252,""name"":""Graphic Novels"",""analytics_n...",3934,MX,Mexico,1652102642,"{""id"":795948953,""name"":""Pura Pinche Fortaleza ...",MXN,$,...,successful,1663713207,0.048703,"{""web"":{""project"":""https://www.kickstarter.com...",0.050213,3816.301584,domestic,"{""id"":1176822,""status"":""successful"",""hls"":""htt...",Pura Pinche Fortaleza Cómics,https://www.kickstarter.com/projects/purapinch...
1,65,The Floasis is a flow and circus arts space th...,"{""id"":256,""name"":""Spaces"",""analytics_name"":""Sp...",6095,US,the United States,1432012582,"{""id"":479379597,""name"":""Tara McManus"",""is_regi...",USD,$,...,successful,1436072992,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,6095.000000,domestic,"{""id"":540971,""status"":""successful"",""hls"":null,...",Tara McManus,https://www.kickstarter.com/projects/479379597...


In [60]:
df['url_scrape'][0]

'https://www.kickstarter.com/projects/purapinchefortaleza/3-novelas-graficas-mexicanas-de-pura-pinche-fortaleza-comics?ref=discovery_category_newest'

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [62]:
def get_story(url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#react-campaign > section > div > nav > li.mb3.border-left.border-left3px.pl2.ml-2.border-left-create-500 > a > strong")))
    page_source = driver.page_source
    driver.quit()

    tree = html.fromstring(page_source)
    xpath = '//*[@id="react-campaign"]/section/div/div/div/div[1]/div/div' 
    paragraphs = tree.xpath(xpath)
    texts = [p.text_content().strip() for p in paragraphs if p.text_content().strip()]
    
    return(texts) #list

In [63]:
test = get_story(df['url_scrape'][0])

In [67]:
df['story'] = df['url_scrape'].apply(get_story)

In [68]:
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,state_changed_at,static_usd_rate,urls,usd_exchange_rate,usd_pledged,usd_type,video,creator_name,url_scrape,story
0,53,#NoEstamosAquídeVisita Publicaremos las nuevas...,"{""id"":252,""name"":""Graphic Novels"",""analytics_n...",3934,MX,Mexico,1652102642,"{""id"":795948953,""name"":""Pura Pinche Fortaleza ...",MXN,$,...,1663713207,0.048703,"{""web"":{""project"":""https://www.kickstarter.com...",0.050213,3816.301584,domestic,"{""id"":1176822,""status"":""successful"",""hls"":""htt...",Pura Pinche Fortaleza Cómics,https://www.kickstarter.com/projects/purapinch...,[Story\nNosotros somos Pura Pinche Fortaleza C...
1,65,The Floasis is a flow and circus arts space th...,"{""id"":256,""name"":""Spaces"",""analytics_name"":""Sp...",6095,US,the United States,1432012582,"{""id"":479379597,""name"":""Tara McManus"",""is_regi...",USD,$,...,1436072992,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,6095.000000,domestic,"{""id"":540971,""status"":""successful"",""hls"":null,...",Tara McManus,https://www.kickstarter.com/projects/479379597...,[StoryHelp us improve the Floasis!Who are we a...
2,70,BILINGUAL Space horror thriller. Novela de cie...,"{""id"":47,""name"":""Fiction"",""analytics_name"":""Fi...",5714,FR,France,1599164730,"{""id"":1801410745,""name"":""Diego Pozo"",""is_regis...",EUR,€,...,1674512581,1.064601,"{""web"":{""project"":""https://www.kickstarter.com...",1.089496,5583.832560,domestic,"{""id"":1201746,""status"":""successful"",""hls"":""htt...",Diego Pozo,https://www.kickstarter.com/projects/180141074...,[Story\n\n\n\n\n\n\n\n¡Meta Alcanzada!\nGracia...
3,15,"I am covering ""Piggy"" by NIN and I need help f...","{""id"":42,""name"":""Pop"",""analytics_name"":""Pop"",""...",270,US,the United States,1323831965,"{""id"":1315244566,""name"":""Joseph Liedtke"",""slug...",USD,$,...,1351447913,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,270.000000,domestic,"{""id"":165332,""status"":""successful"",""hls"":null,...",Joseph Liedtke,https://www.kickstarter.com/projects/joejoekey...,[StoryThe idea for this project wasn't even mi...
4,25,These songs have been in recording purgatory f...,"{""id"":42,""name"":""Pop"",""analytics_name"":""Pop"",""...",476,US,the United States,1348003743,"{""id"":1119744782,""name"":""Dominique LeJeune"",""i...",USD,$,...,1350702031,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,476.500000,domestic,NaN,Dominique LeJeune,https://www.kickstarter.com/projects/111974478...,"[StoryTwo years ago in September 2010, I recor..."


In [ ]:
#for text in texts:
    #print(text)

In [71]:
df.to_csv('Kickstarter_processed_sample.csv')